In [1]:
import numpy as np
import gvar as gv
import sys
sys.path.append('../')
#from h5glance import H5Glance 
import os
import lsqfit
import io
import yaml
import warnings  
warnings.simplefilter(action="default")

In [2]:
os.chdir('../../hyperon-xpt-master/xpt')

In [3]:
import fit_analysis as fa 
import fit_routine as fit
import i_o
import priors

/Users/grantdb/anaconda/lib/python3.6/importlib/_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [4]:
import tests
import importlib
importlib.reload(tests)
importlib.reload(fit)
verbose_fitfcn = tests.VerboseFitfcn(force_gvars=True)
#print(verbose_fitfcn)

In [17]:
from importlib import reload
reload(i_o)
#reload(fa)
reload(fit)

<module 'fit_routine' from '/Users/grantdb/hyperon-xpt-master/xpt/fit_routine.py'>

In [6]:
file = '../data/hyperon_data.h5'
#H5Glance(file)
#file2 = '../data/input_data.h5'
#H5Glance(file2)
#file2 = '~/Documents/hyperon_ga/hyper_spec_axial.h5'

In [7]:
# a_fm = gv.load('a_fm_results.p')
# a_fm_out = gv.evalcorr([v for k,v in a_fm.items()])
# #print(a_fm_out)

In [8]:
# w0_a = gv.load('w0_a_results.p')
# w0_a_out = gv.evalcorr([v for k,v in w0_a.items()])
# print(w0_a_out)

In [9]:
from priors import prior
prior = prior
#print(prior)

In [10]:
f = open('models.yaml', 'r')
models = yaml.load(f)
    #print(keys)
model_info = {}
for item in models.values():
        model_info = item['xi:xi_st:s_n2lo:d_n2lo:x_n2lo']
#print(model_info)

In [11]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_imp')
prior = input_output.make_prior(data,prior)
phys_point_data = input_output.get_data_phys_point()

/Users/grantdb/hyperon-xpt-master/xpt/i_o.py:106: UserWarning: sample sizes differ for different entries: 5000 5001
  gv_data[ens] = gv.dataset.avg_data(gv_data[ens], bstrap=True)


In [12]:
 fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)

In [13]:
print(fit_test.fit)

Least Square Fit:
  chi2/dof [dof] = 0.51 [34]    Q = 0.99    logGBF = -151.63

Parameters:
             m_{xi,0}       1317 (21)      [   1000 (1000) ]  
          m_{xi_st,0}       1540 (28)      [   1300 (1000) ]  
         m_{lambda,0}     1000 (1000)      [   1000 (1000) ]  
          m_{sigma,0}     1200 (1000)      [   1200 (1000) ]  
       m_{sigma_st,0}     1400 (1000)      [   1400 (1000) ]  
          m_{omega,0}     1650 (1000)      [   1650 (1000) ]  
               s_{xi}       2.0 (1.2)      [     0.0 (5.0) ]  
           s_{xi,bar}       1.5 (1.2)      [     0.0 (5.0) ]  
           s_{lambda}     -2e-16 +- 5      [     0.0 (5.0) ]  
        s_{sigma,bar}      6e-17 +- 5      [     0.0 (5.0) ]  
            s_{sigma}      3e-16 +- 5      [     0.0 (5.0) ]  
        s_{omega,bar}     -4e-16 +- 5      [     0.0 (5.0) ]  
            g_{xi,xi}       0.1 (2.9)      [     0.3 (3.0) ]  
         g_{xi_st,xi}       0.7 (1.4)      [     0.7 (3.0) ]  
      g_{xi_st,xi_st}     

In [14]:
p = input_output.get_posterior(fit_test=fit_test,prior=prior,param=None)
fit_post = fit.fit_routine(prior=p, data=phys_point_data,model_info=model_info)
print(fit_post.fit)
#print(p)

Least Square Fit:
  chi2/dof [dof] = 0.93 [2]    Q = 0.39    logGBF = -10.467

Parameters:
             m_{xi,0}          1303 (18)        [    1317 (21) ]  
          m_{xi_st,0}          1512 (18)        [    1540 (28) ]  *
         m_{lambda,0}        1000 (1000)        [  1000 (1000) ]  
          m_{sigma,0}        1200 (1000)        [  1200 (1000) ]  
       m_{sigma_st,0}        1400 (1000)        [  1400 (1000) ]  
          m_{omega,0}        1650 (1000)        [  1650 (1000) ]  
               s_{xi}          2.6 (1.1)        [    2.0 (1.2) ]  
           s_{xi,bar}          2.44 (99)        [    1.5 (1.2) ]  
           s_{lambda}         8e-17 +- 5        [  -2e-16 +- 5 ]  
        s_{sigma,bar}        -1e-16 +- 5        [   6e-17 +- 5 ]  
            s_{sigma}        -1e-14 +- 5        [   3e-16 +- 5 ]  
        s_{omega,bar}        -2e-15 +- 5        [  -4e-16 +- 5 ]  
            g_{xi,xi}          1.45 (59)        [    0.1 (2.9) ]  
         g_{xi_st,xi}          0.9 (1

In [15]:
import mass_relations as mass 
reload(mass)

hyp_mass = mass.Mass_Combinations(lam= p['m_{lambda,0}'], sigma=p['m_{sigma,0}'], xi=p['m_{xi,0}'], delta=p['m_{xi_st,0}'], sigma_st=p['m_{sigma_st,0}'], xi_st=p['m_{xi_st,0}'], omega= p['m_{omega,0}'],p=prior )
#print(mass.m2, mass.m2_c)
print(hyp_mass.eps)
print('Label  Mass_combo')
for label, mass_combs in hyp_mass.M.items():
    print('{}    {}'.format(label, mass_combs))

print('c  Value')
for c, valuee in hyp_mass.c.items():
    print('{}    {}'.format(c, valuee))


print('R  Value')
for r, value in hyp_mass.R.items():
    print('{}    {}'.format(r, value))

print(mass.plot_baryon_mass_mpi)

[1.1893(67)e-07 1.6830(84)e-07 1.4738(57)e-07 1.1706(39)e-07 1.0504(44)e-07
 8.750(32)e-08 1.6501(66)e-07 1.4330(65)e-07 1.1673(47)e-07 1.0396(52)e-07
 8.794(34)e-08 1.7386(55)e-07 1.4632(40)e-07 1.2060(38)e-07 1.1911(46)e-07
 1.0822(35)e-07 9.128(34)e-08]
Label  Mass_combo
M_1    228(79)
M_2    46(16)
M_3    -1.1(1.6)
M_4    -0.03(32)
M_5    0.09(95)
M_6    0.1(1.1)
M_7    0.02(22)
M_8    0.003(32)
M_A    -0.002(141)
M_B    -0.0007(125)
M_C    -0.0002(87)
M_D    -0.002(16)
c  Value
c^{1,0}_0    0.43(15)
c^{1,0}_2    -0.114(40)
c^{8,0}_1    [-27136(38391) -19176(27130) -21898(30980) -27571(39006) -30725(43469)
 -36884(52181) -19559(27671) -22522(31864) -27649(39116) -31044(43919)
 -36698(51919) -18563(26263) -22057(31205) -26762(37862) -27097(38335)
 -29824(42193) -35358(50023)]
c^{8,0}_2    [10011(92107) 7074(65090) 8078(74328) 10171(93582) 11335(104290)
 13607(125193) 7215(66387) 8309(76447) 10200(93848) 11452(105371)
 13538(124564) 6848(63010) 8137(74866) 9873(90838) 9996(91974)
 11

AttributeError: module 'mass_relations' has no attribute 'plot_baryon_mass_mpi'

In [20]:
pdg = {}
pdg['lambda'] = input_output.get_data_phys_point(param='m_lambda')#gv.gvar(1115.683,0.006)
pdg['sigma'] = input_output.get_data_phys_point(param='m_sigma')#Gv.gvar(1192.642, 0.024)
pdg['sigma_st'] = input_output.get_data_phys_point(param='m_sigma_st')#gv.gvar(1382.80,0.35)
pdg['xi'] = input_output.get_data_phys_point(param='m_xi_st') #gv.gvar(1314.86,0.20)
pdg['xi_st'] = input_output.get_data_phys_point(param='m_xi_st')
pdg['delta'] = gv.gvar(1115.683,0.006)
pdg['omega'] = input_output.get_data_phys_point(param='m_omega')

In [21]:
# using pdg masses #
mass_pdg = mass.Mass_Combinations(lam= pdg['lambda'], sigma=pdg['sigma'], xi=pdg['xi'], delta=pdg['delta'], sigma_st=pdg['sigma_st'], xi_st=pdg['xi_st'],p=p )
#print(mass_pdg)
print('Label  Mass_combo')
for label, mass_combs in mass_pdg.M.items():
    print('{}    {}'.format(label, mass_combs))

Label  Mass_combo
M_1    194.096(17)
M_2    38.8191(34)
M_3    -4.29587(68)
M_4    -0.092944(35)
M_5    0.48740(27)
M_6    -0.51503(24)
M_7    -0.103007(48)
M_8    0.015605(16)
M_A    0.019141(62)
M_B    -0.0053808(26)
M_C    -0.0008227(21)
M_D    -0.0078026(78)


In [41]:
from importlib import reload
reload(fa)
fit_analysis = fa.fit_analysis(phys_point_data=phys_point_data, data=data,model_info=model_info, prior=prior)
hi = fit_analysis.error_budget
print(hi)

/Users/grantdb/hyperon-xpt-master/xpt/fit_analysis.py:556: DeprecationWarning: invalid escape sequence \p
  label = '$m_\pi$'
dict_values([1000(1000), 1200(1000), 1400(1000), 1000(1000), 1300(1000), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.3(3.0), 0.7(3.0), -0.5(3.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(5.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0.0(1.0), 0(0), 0(0), 0(0), inf(0), 197.327(0), 1157.0(7.2), 134.80(30), 494.20(30), 1115.6830(60), 1193.154(27), 1384.57(39), 1318.28(11), 1533.40(34), 1672(29), array([561.9(5.0), 493.9(4.0), 517.0(4.2), 544.9(4.4), 558.2(4.5),


In [29]:
hi = fit_analysis.error_budget
print(hi)

dict_keys(['m_{lambda,0}', 'm_{sigma,0}', 'm_{sigma_st,0}', 'm_{xi,0}', 'm_{xi_st,0}', 'd_{lambda,a}', 'd_{sigma,a}', 'd_{sigma_st,a}', 'd_{xi,a}', 'd_{xi_st,a}', 'd_{lambda,aa}', 'd_{lambda,al}', 'd_{sigma,aa}', 'd_{sigma,al}', 'd_{sigma_st,aa}', 'd_{sigma_st,al}', 'd_{xi,aa}', 'd_{xi,al}', 'd_{xi_st,aa}', 'd_{xi_st,al}', 's_{lambda}', 's_{sigma}', 's_{sigma,bar}', 's_{xi}', 's_{xi,bar}', 'g_{lambda,sigma}', 'g_{lambda,sigma_st}', 'g_{sigma,sigma}', 'g_{sigma_st,sigma}', 'g_{sigma_st,sigma_st}', 'g_{xi,xi}', 'g_{xi_st,xi}', 'g_{xi_st,xi_st}', 'b_{lambda,4}', 'b_{sigma,4}', 'b_{sigma_st,4}', 'b_{xi,4}', 'b_{xi_st,4}', 'a_{lambda,4}', 'a_{sigma,4}', 'a_{sigma_st,4}', 'a_{xi,4}', 'a_{xi_st,4}', 'd_{lambda,s}', 'd_{sigma,s}', 'd_{sigma_st,s}', 'd_{xi,s}', 'd_{xi_st,s}', 'd_{lambda,as}', 'd_{lambda,ls}', 'd_{lambda,ss}', 'd_{sigma,as}', 'd_{sigma,ls}', 'd_{sigma,ss}', 'd_{sigma_st,as}', 'd_{sigma_st,ls}', 'd_{sigma_st,ss}', 'd_{xi,as}', 'd_{xi,ls}', 'd_{xi,ss}', 'd_{xi_st,as}', 'd_{xi_st,l

In [59]:
sorted(hi.items())

[('disc',
  'Partial % Errors:\n                              particle\n--------------------------------------\n        m_{lambda,0} [disc]:      0.00\n         m_{sigma,0} [disc]:      0.00\n      m_{sigma_st,0} [disc]:      0.00\n            m_{xi,0} [disc]:      0.02\n         m_{xi_st,0} [disc]:      0.02\n        d_{lambda,a} [disc]:      0.00\n         d_{sigma,a} [disc]:      0.00\n      d_{sigma_st,a} [disc]:      0.00\n            d_{xi,a} [disc]:      0.14\n         d_{xi_st,a} [disc]:      0.10\n       d_{lambda,aa} [disc]:      0.00\n       d_{lambda,al} [disc]:      0.00\n        d_{sigma,aa} [disc]:      0.00\n        d_{sigma,al} [disc]:      0.00\n     d_{sigma_st,aa} [disc]:      0.00\n     d_{sigma_st,al} [disc]:      0.00\n           d_{xi,aa} [disc]:      0.44\n           d_{xi,al} [disc]:      0.18\n        d_{xi_st,aa} [disc]:      0.29\n        d_{xi_st,al} [disc]:      0.11\n           s_{lambda} [xpt]:      0.00\n            s_{sigma} [xpt]:      0.00\n        

In [ ]:
for j, ens_j in enumerate(fit_analysis.ensembles):
    print(j,ens_j)

In [ ]:
print(fit_analysis.fitfcn(p=p,data=phys_point_data,particle=None))

In [19]:
print(fit_analysis.posterior)

{'m_{xi,0}': 1320(18), 'm_{xi_st,0}': 1552(24), 'm_{lambda,0}': 1108(20), 'm_{sigma,0}': 1204(17), 'm_{sigma_st,0}': 1457(28), 's_{xi}': 1.6(1.0), 's_{xi,bar}': 1.1(1.1), 's_{lambda}': 2.09(85), 's_{sigma,bar}': 0.13(98), 's_{sigma}': 2.01(91), 'g_{xi,xi}': 0.09(2.87), 'g_{xi_st,xi}': 0.6(1.5), 'g_{xi_st,xi_st}': -0.5(2.0), 'g_{lambda,sigma}': 8e-18 +- 5, 'g_{lambda,sigma_st}': 0.66(61), 'g_{sigma,sigma}': -2e-16 +- 5, 'g_{sigma_st,sigma}': 0.7(1.5), 'g_{sigma_st,sigma_st}': 0.0004 +- 5, 'a_{xi,4}': -0.9(4.2), 'a_{xi_st,4}': 0.4(4.5), 'b_{xi,4}': -0.7(4.6), 'b_{xi_st,4}': 0.9(4.6), 'a_{sigma,4}': -0.8(4.3), 'a_{sigma_st,4}': -2.1(3.8), 'b_{sigma,4}': 0.1(4.6), 'b_{sigma_st,4}': 0.7(4.4), 'a_{lambda,4}': -0.6(4.1), 'b_{lambda,4}': 0.6(4.4), 'd_{xi,a}': -0.45(13), 'd_{xi_st,a}': -0.10(16), 'd_{xi,s}': 0.43(34), 'd_{xi_st,s}': 0.01(37), 'd_{lambda,s}': -0.25(42), 'd_{lambda,a}': -0.44(16), 'd_{sigma_st,a}': 0.07(21), 'd_{sigma_st,s}': -0.18(43), 'd_{sigma,s}': 0.67(39), 'd_{sigma,a}': -0.

In [20]:
print(fit_analysis._extrapolate_to_ens(ens=None,phys_params=None))

{'a09m135': {'xi': 1304.9(7.1), 'xi_st': 1555.8(9.9), 'lambda': 1106.3(8.1), 'sigma': 1213.9(6.8), 'sigma_st': 1466(12)}, 'a09m220': {'xi': 1326.9(5.9), 'xi_st': 1579.6(8.4), 'lambda': 1152.9(5.9), 'sigma': 1239.4(5.8), 'sigma_st': 1486(10)}, 'a09m310': {'xi': 1359.2(5.3), 'xi_st': 1614.0(7.5), 'lambda': 1219.2(4.9), 'sigma': 1279.6(5.3), 'sigma_st': 1526.7(8.9)}, 'a09m350': {'xi': 1375.7(5.3), 'xi_st': 1630.4(7.5), 'lambda': 1248.3(5.0), 'sigma': 1301.4(5.4), 'sigma_st': 1548.7(8.8)}, 'a09m400': {'xi': 1398.9(7.0), 'xi_st': 1654.3(9.1), 'lambda': 1289.4(6.8), 'sigma': 1332.9(7.0), 'sigma_st': 1584(11)}, 'a12m130': {'xi': 1268.1(5.8), 'xi_st': 1535.7(7.5), 'lambda': 1082.9(6.5), 'sigma': 1188.9(5.8), 'sigma_st': 1465.6(8.0)}, 'a12m220L': {'xi': 1289.0(4.6), 'xi_st': 1557.8(5.7), 'lambda': 1128.4(4.4), 'sigma': 1211.3(4.4), 'sigma_st': 1481.9(5.8)}, 'a12m310': {'xi': 1321.9(4.2), 'xi_st': 1588.9(5.2), 'lambda': 1189.3(3.9), 'sigma': 1250.9(4.2), 'sigma_st': 1514.9(5.8)}, 'a12m350': {'xi

In [24]:
x = fit_analysis.fitter.y
print(x)


{'xi': array([1282(12), 1318(11), 1370(11), 1388(11), 1411(12), 1271.5(9.6),
       1283(10), 1318.6(9.9), 1336(10), 1358(10), 1258.0(8.6),
       1257.6(8.5), 1294.3(8.7), 1288.1(9.1), 1302.1(9.7), 1327.5(9.8)],
      dtype=object), 'xi_st': array([1527(16), 1563(19), 1620(21), 1631(27), 1671(15), 1535(13),
       1555(12), 1581(13), 1599(14), 1622(13), 1528(11), 1518(13),
       1557(10), 1554(11), 1555(13), 1584(13)], dtype=object), 'sigma': array([1201(10), 1215(13), 1291(11), 1312(12), 1347(12), 1186.3(9.6),
       1208.7(9.2), 1248.0(9.6), 1275.2(9.7), 1298(10), 1206(10),
       1194.5(8.9), 1236.8(8.3), 1229.1(9.2), 1237.6(9.5), 1280.5(9.0)],
      dtype=object), 'sigma_st': array([1423(23), 1461(23), 1541(26), 1573(22), 1601(17), 1462(14),
       1480(12), 1507(17), 1536(16), 1558(17), 1485(13), 1447(23),
       1499(10), 1499(11), 1490(21), 1537(13)], dtype=object), 'lambda': array([1062(26), 1142(11), 1229(10), 1260(10), 1304(11), 1090(11),
       1123.8(9.5), 1187.4(9.1), 12